In [3]:
%cd ..

/Users/seungwoo/Projects/wos-project/wos


In [6]:
%cd wos

/Users/seungwoo/Projects/wos-project/wos


In [9]:
from pathlib import Path

import igl
import numpy as np

from src.utils.geometry_utils import normalize_mesh

In [19]:
dragon_path = "../data/xyzrgb_dragon.obj"
bunny_path = "../data/stanford-bunny.obj"
method = "unit_cube"
# assert dragon_path.exists(), "Could not find dragon mesh"

v, f = igl.read_triangle_mesh(str(bunny_path))
v_ = normalize_mesh(v, method=method)

print(v_.min(axis=0), v_.max(axis=0))

igl.write_triangle_mesh(f"../data/stanford-bunny_{method}.obj", v_, f)

[-0.5 -0.5 -0.5] [0.49999994 0.49999994 0.49999992]


True

In [33]:
# v, f = igl.read_triangle_mesh(f"../data/stanford-bunny_{method}.obj")
v, f = igl.read_triangle_mesh(f"../data/xyzrgb_dragon_{method}.obj")

img_width = 512
img_height = 512
z = 0.5

xs = np.linspace(-0.5, 0.5, img_width)
ys = np.linspace(-0.5, 0.5, img_height)
xx, yy = np.meshgrid(xs, ys, indexing="xy")

# Flatten the query points
query_pts = np.stack(
    [xx.flatten(), yy.flatten(), np.full_like(xx.flatten(), z)],
    axis=1,
)
query_pts = query_pts.reshape(-1, 3).astype(np.float32)

# Compute the signed distance
d, _, _ = igl.signed_distance(query_pts, v, f)
d = d.reshape(img_height, img_width)

In [39]:
import matplotlib.pyplot as plt

for z in np.linspace(-0.5, 0.5, 20):
    query_pts = np.stack(
        [xx.flatten(), yy.flatten(), np.full_like(xx.flatten(), z)],
        axis=1,
    )
    query_pts = query_pts.reshape(-1, 3).astype(np.float32)

    d, _, _ = igl.signed_distance(query_pts, v, f)
    d = d.reshape(img_height, img_width)

    mask = d > 0
    d = np.where(mask, 255, 0)
    d = d[..., None].repeat(3, axis=-1)

    plt.figure()
    # plt.imshow(d, cmap="coolwarm")
    plt.imshow(d)
    plt.savefig(f"../data/signed_distance_{z}.png")
    plt.close()